In [4]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
spark = SparkSession.builder.appName('pivot').getOrCreate()

data = [("Fat",1000,"Study1"), ("Carbohydrate",1500,"Study1"), ("Beans",1600,"Study1"), \
      ("Protein",2000,"Study1"),("Protein",2000,"Study1"),("Fat",400,"Study2"), \
      ("Carbohydrate",1200,"Study2"),("Minerals",1500,"Study2"),("Protein",4000,"Study2"), \
      ("Fat",2000,"Study3"),("Carbohydrate",2000,"Study3"),("Beans",2000,"Study4")]

columns= ["nutriment","amount","study"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)


root
 |-- nutriment: string (nullable = true)
 |-- amount: long (nullable = true)
 |-- study: string (nullable = true)

+------------+------+------+
|nutriment   |amount|study |
+------------+------+------+
|Fat         |1000  |Study1|
|Carbohydrate|1500  |Study1|
|Beans       |1600  |Study1|
|Protein     |2000  |Study1|
|Protein     |2000  |Study1|
|Fat         |400   |Study2|
|Carbohydrate|1200  |Study2|
|Minerals    |1500  |Study2|
|Protein     |4000  |Study2|
|Fat         |2000  |Study3|
|Carbohydrate|2000  |Study3|
|Beans       |2000  |Study4|
+------------+------+------+



In [5]:

pivotDF = df.groupBy("nutriment").pivot("study").sum("amount")
pivotDF.printSchema()
pivotDF.show(truncate=False)

pivotDF = df.groupBy("nutriment","study") \
      .sum("amount") \
      .groupBy("nutriment") \
      .pivot("study") \
      .sum("sum(amount)")
pivotDF.printSchema()
pivotDF.show(truncate=False)


root
 |-- nutriment: string (nullable = true)
 |-- Study1: long (nullable = true)
 |-- Study2: long (nullable = true)
 |-- Study3: long (nullable = true)
 |-- Study4: long (nullable = true)

+------------+------+------+------+------+
|nutriment   |Study1|Study2|Study3|Study4|
+------------+------+------+------+------+
|Beans       |1600  |null  |null  |2000  |
|Carbohydrate|1500  |1200  |2000  |null  |
|Fat         |1000  |400   |2000  |null  |
|Protein     |4000  |4000  |null  |null  |
|Minerals    |null  |1500  |null  |null  |
+------------+------+------+------+------+

root
 |-- nutriment: string (nullable = true)
 |-- Study1: long (nullable = true)
 |-- Study2: long (nullable = true)
 |-- Study3: long (nullable = true)
 |-- Study4: long (nullable = true)

+------------+------+------+------+------+
|nutriment   |Study1|Study2|Study3|Study4|
+------------+------+------+------+------+
|Beans       |1600  |null  |null  |2000  |
|Carbohydrate|1500  |1200  |2000  |null  |
|Fat         |1

In [6]:


""" unpivot """
""" unpivot """
unpivotExpr = "stack(3, 'Study4', Study4, 'Study3', Study3, 'Study2', Study2) as (study,total)"
unPivotDF = pivotDF.select("nutriment", expr(unpivotExpr)) \
    .where("Total is not null")
unPivotDF.show(truncate=False)

+------------+-------+-----+
|nutriment   |studies|Total|
+------------+-------+-----+
|Beans       |Study4 |2000 |
|Carbohydrate|Study3 |2000 |
|Carbohydrate|Study2 |1200 |
|Fat         |Study3 |2000 |
|Fat         |Study2 |400  |
|Protein     |Study2 |4000 |
|Minerals    |Study2 |1500 |
+------------+-------+-----+

